# Fractional Flow

###  This app plots fractional flow and it's derivative 

#### The Buckley Leverett Equation

In the Buckley Leverett (BL) solution for linear displacement each water saturation moves through the rock at a velocity that is computed from the derivative of the fractional flow with respect to water saturation. The BL solution for 1-D, 2-phase (oil and water) flow, assuming no capillary pressure after simplification is:
<br />
<br />

$$ \frac{\partial S_W}{\partial t_d} + (\frac{\partial f_W}{\partial S_W} ) \frac{\partial S_W}{\partial x_d} = 0 $$
<br />
<br />

The term in the parenthesis is the derivative of fractional flow. Fractional flow of water can be described as:

$$ f_W = \frac {u_W}{u_W + u_O} $$
<br />
<br />


where $u$ is the velocity with which each phase moves through the porous medium. Using relations form relative permeability and **assuming no capillary pressure** this equation can be re-written as:
<br />
<br />

$$ f_W = \frac {1 + \frac{k \; k_{ro}}{u \; \mu_O}(-Δ\rho g \; sin\alpha)}{1+ \frac{k_{ro} \; \mu_W}{k_{rw} \; \mu_O}} $$
<br />
<br />

where:<br />
$k$ = average permeability in mD <br />
$u$ = average velocity in $\frac {ft}{day}$
<br /> $\mu_O$ = viscosity of oil in cp 
<br /> $\mu_W$ = viscosity of water = 1 cp  <br />
Δ$\rho$ = $\rho_W - \rho_O \;\; (\rho_W = 62.4 \frac{lb}{ft^3})$  <br />
$krw_o$ = endpoint relative permeability for water <br /> $kro_o$ = endpoint relative permeability for oil
<br /> 
<br /> 
The derivative is estimated numerically by choosing a slightly perturbed saturation $S_{W2}=S_{W1} + 0.00001$, and using finite differences:

$$ f_W = \frac{f_{W2}-f_{W1}}{S_{W2}-S_{W1}} $$
<br /> 
<br /> 
You can change the parameters below to see how each of them affect the plots. <br />$n_o$ and $n_w$ are the Corey exponents used for the underlying relative permeability curves.
<br /> 
<br />

In [2]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from ipywidgets import BoundedFloatText,  interact,FloatSlider
%matplotlib inline
    
def fracflow(Swr,Sor,krwo,kroo,visco,u,alpha,rhoo,k,no,nw):
    
    viscw = 1 # cp
    rhow = 62.4 # lb/ft3
    
    Swr2 = 1-Sor

    Sw = np.linspace(Swr,Swr2,100)
    Sw1 = Sw + 0.0001

    krw = np.empty(100)
    kro = np.empty(100) 
    krw1 = np.empty(100)
    kro1 = np.empty(100)
    
    for i,j in enumerate(Sw):
        
        S = (j-Swr)/(1-Swr-Sor)
        krw[i] = krwo*(S**nw)
        kro[i] = kroo*((1-S)**no)
         
    fw = np.empty(100)
    dfw = np.empty(100)
    
    for i,j in enumerate(Sw):
        
        with np.errstate(divide='ignore'):
        
            bottom = (1+(kro[i]*viscw)/(krw[i]*visco))

            fw[i]=(1+(6.33E-3/144)*((k*kro[i])/(u*visco))*((rhoo-rhow)*m.sin(np.deg2rad(alpha))))/bottom
                   
            
            if i == 0 :
                
                dfw[i] = 0
                
            else:
                
                dfw[i]=(fw[i]-fw[i-1])/(Sw[i]-Sw[i-1])
                
    
    f = plt.figure(figsize=(15,5))
    ax = f.add_subplot(121)
    ax2 = f.add_subplot(122)
    ax.plot(Sw,fw,'b-');
    ax.set_title('Fractional Flow of Water')
    ax.set_xlabel('$S_W$')
    ax.set_ylabel('$f_W$')
    ax.grid()
    ax2.plot(Sw,dfw,'g-');
    ax2.set_title('Derivative of Fractional Flow')
    ax2.set_xlabel('$S_W$')
    ax2.set_ylabel('$df_W/dS_W$')
    ax2.grid() 
    
InputSwr = BoundedFloatText(value=0.25,min=0,max=1,step=0.01,description='S$_{wr}$',disabled=False)
InputSor = BoundedFloatText(value=0.3,min=0,max=1,step=0.01,description='S$_{or}$',disabled=False)
Inputkrwo = BoundedFloatText(value=0.2,min=0,max=1,step=0.01,description='krw$o$',disabled=False)
Inputkroo = BoundedFloatText(value=1,min=0,max=1,step=0.01,description='kro$o$',disabled=False)
Inputvisco = BoundedFloatText(value=5,min=0,max=150,step=0.1,description='Visc [cp]',disabled=False)
Inputu = BoundedFloatText(value=0.01,min=0,max=100,step=0.01,description='Vel. [ft/day]',disabled=False)
Inputalpha = BoundedFloatText(value=0,min=-180,max=180,step=0.01,description='Angle [deg]',disabled=False)
Inputrhoo = BoundedFloatText(value=40.5,min=0,max=62.4,step=0.01,description='$ρ_o$ [lb/ft^3]',disabled=False)
Inputk = BoundedFloatText(value=416,min=0,max=1000,step=0.01,description='Perm [mD]',disabled=False)

interact (fracflow,Swr=InputSwr, 
          Sor=InputSor,
          krwo=Inputkrwo,
          kroo=Inputkroo,
          visco=Inputvisco,
          u=Inputu,
          alpha=Inputalpha,
          rhoo=Inputrhoo,
          k=Inputk,
                 
          no=FloatSlider(value=2,min=1,max=6.0,step=0.01,disabled=False,continuous_update=False,description='$n_o$',
                             orientation='horizontal',
                             layout = {'width':'40%'},
                             readout=True,
                             readout_format='.2f'),
          nw=FloatSlider(value=2,min=1,max=6.0,step=0.01,disabled=False,continuous_update=False,description='$n_w$',
                             orientation='horizontal',
                             layout = {'width':'40%'},
                             readout=True,
                             readout_format='.2f'));


interactive(children=(BoundedFloatText(value=0.25, description='S$_{wr}$', max=1.0, step=0.01), BoundedFloatTe…

<br />
<br />
<br />
<br />
<br />

Designed by **Fahim N. Is.** , *BS. Petroleum Engineering, The University of Texas at Austin*